In [9]:
import cv2
import numpy as np
import os
import math

# Technique 1 – Running Average

The concept of running average is to detect active objects and remove them i.e. differentiate between pixels that seem to change over time and remove them. Here “Running” signifies the fact that the average is being computed over previous and current frames again and again until the frames are exhausted. Simply, the background is the mean of ‘n’ previous frames in the video.



In [31]:

def calculate_running_average(ROOT, file_path, num_bins):

    cap = cv2.VideoCapture(file_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the number of frames per bin
    frames_per_bin = math.ceil(num_frames / num_bins)
    image_num = 0
    for i in range(num_bins):
        start_frame = i * frames_per_bin
        end_frame = min((i + 1) * frames_per_bin - 1, num_frames - 1)

        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        current_frame = start_frame
        first_iter = True

        while True:
            ret, frame = cap.read()

            if frame is None or current_frame > end_frame:
                break

            if first_iter:
                avg = np.float32(frame)
                first_iter = False

            cv2.accumulateWeighted(frame, avg, 0.005)
            current_frame += 1

        result = cv2.convertScaleAbs(avg)
        outfile_name = f"running_average_{start_frame}_{end_frame}.jpg"
        # we're going to save the jpg based on starting frame
        cv2.imwrite(f"{ROOT}/{image_num}.jpg", result)
        image_num += 1

    cap.release()
    return fps

def create_video_from_images(image_folder, output_path, fps):
    image_files = [file for file in os.listdir(image_folder) if file.endswith(".jpg")]
    image_files = sorted(image_files, key=lambda x: int(os.path.splitext(x)[0]))
    print(image_files)
    image_path = os.path.join(image_folder, image_files[0])
    img = cv2.imread(image_path)
    height, width, _ = img.shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for i, image_file in enumerate(image_files):
        image_path = os.path.join(image_folder, image_file)
        img = cv2.imread(image_path)
        out.write(img)

    out.release()

# Use the function to calculate the running averages over different frame ranges
file_path = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080922_extinction_1/USB camera/hSyn-AS-Gi-1-220809-093642_Cam1.avi"
OUT_ROOT = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080922_extinction_1/running_avgs"

fps = calculate_running_average(OUT_ROOT, file_path, num_bins=35)

# Call the function to create the video
create_video_from_images(OUT_ROOT, OUT_ROOT, fps)


['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg']


[ERROR:0] global /home/conda/feedstock_root/build_artifacts/libopencv_1633502019957/work/modules/videoio/src/cap.cpp (587) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.3) /home/conda/feedstock_root/build_artifacts/libopencv_1633502019957/work/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080922_extinction_1/running_avgs in function 'icvExtractPattern'




# Technique 2 – Median filtering
Temporal median filtering is one of a kind background subtraction method that approximates the median of each pixel to be the background. We can make the assumption that the median pixel is background due to the fact that the background is most likely to be in the video the longest. Thus we can apply this logic to frames as well and say that the background frame is equal to the mean of the previous ‘n’ frames. However, median filtering is a computationally heavy and time-consuming process thus its applications are limited.

In [ ]:
video = cv2.VideoCapture(file_path)
FOI = video.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=30)
frames = []
for frameOI in FOI:
    video.set(cv2.CAP_PROP_POS_FRAMES, frameOI)
    ret, frame = video.read()
    frames.append(frame)
result2 = np.median(frames, axis=0).astype(dtype=np.uint8)

In [ ]:
cv2.imshow("Median filtering result",result2) 
cv2.waitKey(0)